 # 서포트 벡터 머신 (SVM)

###  박태영 교수
### 연세대학교 응용통계학과

### Outline

### 4. SVM 최적화 및 성능평가
    4.1 파이프라인
    4.2 k-겹 교차검증
    4.3 그리드 서치를 이용한 SVM 최적화

#### 4.1 파이프라인

- 데이터 전처리 및 모형 학습에 대한 일련의 과정을 하나의 단일 과정으로 만듬
- 교차검증을 수행하고 모형을 최적화하기 위해서 반복되는 작업의 실행을 단순화
- 필요한 모듈을 설치

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches 
from IPython.display import Image
from mlxtend.plotting import plot_decision_regions

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings(action='ignore')

- 반도체 생산 과정에서 590개의 센서로부터 얻은 데이터
- 590개의 숫자형 특성변수, **정상/비정상**에 대한 범주변수(-1:정상,1:비정상)로 구성
- 목적: 새로운 반도체 센서 데이터가 주어졌을 때, 비정상을 예측 분류
- 출처: http://archive.ics.uci.edu/ml/datasets/secom  

In [ ]:
data = pd.read_csv('Secom.csv')
print(data.shape)

- 결측 비율이 10%이상인 특성 변수 제외

In [ ]:
missing = data.isnull().sum() / len(data)
data_com = data.loc[:,[x < 0.1 for x in missing]]

- 결측값의 평균값 대체

In [ ]:
data1 = data2 = data_com
data1.fillna(method = 'ffill', inplace = True) # 이전 시점 값으로 대체
data2.fillna(method = 'bfill', inplace = True) # 이후 시점 값으로 대체
data_com = (data1+data2)/2

- 특성 변수 및 범주 변수의 정의

In [ ]:
X = data_com.iloc[:,:-1] # 특성 변수
y = (data_com.iloc[:,-1]+1)/2 # 범주 변수

- 과대표집(oversampling)을 위해 SMOTE (Synthetic Minority Oversampling Technique) 사용

In [ ]:
Image(filename='Sampling.png', width=500) 

In [ ]:
from imblearn.over_sampling import SMOTE

SMOTE = SMOTE()
X2, y2 = SMOTE.fit_resample(X, y)

In [ ]:
# !pip install imblearn # SMOTE를 사용하기 위해 imblearn 설치

In [ ]:
# !pip install delayed # imblearn 설치를 위해 delayed 설치

- `imblearn`과 `delayed`를 설치한 이후에도 설치가 안되었다고 에러가 발생하는 경우 

In [ ]:
# !pip install ipython # 주피터 노트북의 파이썬 커널 설치

In [ ]:
# !python -m pip install ipykernel # 가상 환경에서 주피터 노트북 설치

In [ ]:
# !python -m ipykernel install --user # 주피터 노트북에 파이썬 커널 등록 후 주피터 노트북 완전히 종료 후 다시 실행

- 훈련용 데이터와 테스트용 데이터를 7:3의 비율로 분리하고 y 비율이 동일하게 함

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.3, random_state=1, stratify=y2)

- `Pipeline`을 통해 전처리 및 모형 학습의 과정을 하나의 과정으로 묶어줌
- 예를 들면, **'표준화 + PCA + SVM'**을 한꺼번에 묶음

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

pipe = make_pipeline(StandardScaler(), # 표준화
                     PCA(n_components=10), # PCA
                     SVC(kernel='rbf', gamma='auto', C=1)) # SVM

pipe.fit(X_train, y_train) # 표준화 + PCA + SVM
y_pred = pipe.predict(X_test) # 예측값
print('Accuracy: %.3f' % pipe.score(X_test, y_test)) # 예측 정확도

#### 4.2 k-겹 교차검증 (k-fold cross validation)

- 머신러닝 알고리즘의 모수값을 디폴트로 정하지 않고 최적화하고자 할 때, 동일한 테스트용 데이터를 이용하면 과적합의 위험이 존재
- 훈련용 데이터를 "훈련용 데이터 + 검증용 데이터"로 나눠 모수값을 최적화하는 데 사용
- 특정 검증용 데이터에 과적합하는 것을 방지하기 위해 여러 겹으로 나누어 검증

In [ ]:
from sklearn.model_selection import cross_val_score

CVS = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=10) # 10-겹 교차검증
print(CVS)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(CVS), np.std(CVS)))

- 정확도 외에 정밀도, 재현율, F1점수를 기준으로 교차검증

In [ ]:
from sklearn.metrics import make_scorer

CVS2 = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=10, scoring=make_scorer(precision_score)) # 정밀도
CVS3 = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=10, scoring=make_scorer(recall_score))    # 재현율 
CVS4 = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=10, scoring=make_scorer(f1_score))        # F1점수

print('CV precision: %.3f +/- %.3f' % (np.mean(CVS2), np.std(CVS2)))
print('CV recall   : %.3f +/- %.3f' % (np.mean(CVS3), np.std(CVS3)))
print('CV F1 score : %.3f +/- %.3f' % (np.mean(CVS4), np.std(CVS4)))

- 훈련용 데이터의 정확도와 교차검증의 정확도 비교
- SVM의 모수 C의 값을 변화하면서 비교
- 모형의 모수값을 나타내는 방법: 모형과 모수 사이에 `__`를 붙여줌 (예: `svc__C`)

In [ ]:
from sklearn.model_selection import validation_curve

pipe = make_pipeline(StandardScaler(),
                     SVC(kernel='rbf', gamma='auto')) 

param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0] # 하나의 모수(C)에 대한 여러 후보값
train_scores, cv_scores = validation_curve(estimator=pipe, 
                                           X=X_train, y=y_train, 
                                           param_name='svc__C', 
                                           param_range=param_range, 
                                           cv=10)

- **과소적합**: 훈련용 데이터의 정확도와 교차검증의 정확도가 모두 낮은 경우
- **과대적합**: 훈련용 데이터의 정확도는 높으나 교차검증의 정확도는 낮은 경우

In [ ]:
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
cv_mean = np.mean(cv_scores, axis=1)
cv_std = np.std(cv_scores, axis=1)

print("Training score: ", train_mean)
print("CV score      : ", cv_mean)

- 검증곡선(validation curve)의 시각화

In [ ]:
plt.plot(param_range, train_mean, color='b', marker='o', markersize=5, 
         label='Training accuracy')

plt.fill_between(param_range, train_mean + train_std, train_mean - train_std, 
                 alpha=0.15, color='b')

plt.plot(param_range, cv_mean, color='g', linestyle='--', marker='s', markersize=5, 
         label='CV accuracy')

plt.fill_between(param_range, cv_mean + cv_std, cv_mean - cv_std, 
                 alpha=0.15, color='g')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.8, 1.0])
plt.savefig('VC.png', dpi=300)
plt.show()

#### 4.3 그리드 서치를 이용한 SVM 최적화

- 여러 모형과 모수의 여러 후보값을 고려하여 머신러닝 알고리즘을 최적화
- 그리드 서치(grid search)를 사용

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe = make_pipeline(StandardScaler(), SVC()) 

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']},
              {'svc__C': param_range, 'svc__kernel': ['rbf'], 'svc__gamma': param_range}]

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10, 
                  n_jobs=-1) # n_jobs=-1 uses all processors
gs = gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_params_)

In [ ]:
svm = gs.best_estimator_
svm.fit(X_train, y_train)
print('Accuracy: %.3f' % svm.score(X_test, y_test))

-  그리드 서치에서 정확도가 아닌 다른 지표(정밀도, 재현율, F1점수)로 최적의 알고리즘을 선택

In [ ]:
#scoring = make_scorer(precision_score) # 정밀도
scoring = make_scorer(recall_score)    # 재현율 
#scoring = make_scorer(f1_score)        # F1점수

pipe = make_pipeline(StandardScaler(), SVC()) 

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']},
              {'svc__C': param_range, 'svc__kernel': ['rbf'], 'svc__gamma': param_range}]

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring=scoring, # 다른 지표를 사용 
                  cv=10, 
                  n_jobs=-1) # n_jobs=-1 uses all processors
gs = gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_params_)

In [ ]:
y_pred = gs.best_estimator_.predict(X_test)

print('Accuracy : %.3f' % accuracy_score(y_true=y_test, y_pred=y_pred))
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred))
print('Recall   : %.3f' % recall_score(y_true=y_test, y_pred=y_pred))
print('F1 score : %.3f' % f1_score(y_true=y_test, y_pred=y_pred))